In [1]:
!pip install seaborn


Defaulting to user installation because normal site-packages is not writeable


In [2]:

%pip install matplotlib
%pip install seaborn
%pip install pandas numpy scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
def balance_check(data, target):
    try:
        df = pd.read_csv(data)
    except:
        print("Error Reading cannot be found")
        return

    #now check value counts
    ones_percentage = df[target].value_counts(normalize=True).loc[1] * 100
    print(f"Percentage of 1s for : {ones_percentage:.2f}%")

    return



In [4]:
balance_check("data/train.csv/train.csv","toxic")
balance_check("data/train.csv/train.csv","severe_toxic")
balance_check("data/train.csv/train.csv","obscene")
balance_check("data/train.csv/train.csv","threat")
balance_check("data/train.csv/train.csv","insult")
balance_check("data/train.csv/train.csv","identity_hate")

Percentage of 1s for : 9.58%
Percentage of 1s for : 1.00%
Percentage of 1s for : 5.29%
Percentage of 1s for : 0.30%
Percentage of 1s for : 4.94%
Percentage of 1s for : 0.88%


In [5]:
df1 = pd.read_csv("data/train.csv/train.csv")
df1.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:

%pip install nltk

%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cpu

%pip install transformers
%pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/nightly/cpu
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import numpy as np
import re
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

#  NLTK Fix for LookupError
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Dynamically resolve the user's nltk_data path
nltk_path = os.path.join(os.path.expanduser('~'), 'AppData', 'Roaming', 'nltk_data')
if nltk_path not in nltk.data.path:
    nltk.data.path.append(nltk_path)

# Trigger a preload of stopwords to catch missing resource issues early
try:
    _ = stopwords.words('english')
except LookupError as e:
    print("NLTK stopwords resource not found, attempting to download again...")
    nltk.download('stopwords')
    _ = stopwords.words('english')

def load_dataset(file_path):
    try:
        df = pd.read_csv(file_path)

        print("Dataset loaded successfully!")
        print(f"Shape: {df.shape}")

        # check for null values
        print("\nNull values per column:")
        print(df.isnull().sum())

        return df

    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # remove artifacts and wikimarkups
    text = re.sub(r'==.*?==+', '', text)
    text = re.sub(r'\[\[.*?\]\]', '', text)
    text = re.sub(r'\{\{.*?\}\}', '', text)
    text = re.sub(r'__[A-Z]+__', '', text)
    text = re.sub(r'&[a-z]+;', '', text)
    text = re.sub(r'<.*?>', '', text)

    # URLs, IPs, timestamps, extra spaces, and lowercasing
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', '', text)
    text = re.sub(r'\d{1,2}:\d{2},?\s*\d{1,2}\s+\w+\s+\d{4}', '', text)
    text = re.sub(r"[^\w\s'!?.]", ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()

    return text

def remove_stopwords(text):
    try:
        stop_words = set(stopwords.words('english'))

        custom_stopwords = {
            'wikipedia', 'page', 'article', 'edit', 'user', 'talk',
            'section', 'redirect', 'category', 'file', 'image', 'template'
        }
        stop_words.update(custom_stopwords)

        negation_words = {
            'not', 'no', 'never', 'none', 'nor', 'neither',
            "don't", "isn't", "aren't", "wasn't", "weren't",
            "haven't", "hasn't", "hadn't", "won't", "wouldn't"
        }
        stop_words -= negation_words

        tokens = word_tokenize(text)
        filtered_tokens = [
            word for word in tokens 
            if word not in stop_words and len(word) > 2 and not word.startswith("'")
        ]

        return ' '.join(filtered_tokens)
    except LookupError as e:
        if "stopwords" in str(e) or "punkt" in str(e):
            nltk.download('punkt', quiet=True)
            nltk.download('stopwords', quiet=True)
            try:
                stop_words = set(stopwords.words('english'))
                tokens = word_tokenize(text)
                return ' '.join(tokens)  # fallback: just tokenize
            except Exception:
                return text  # worst case: return raw text
        return text

def initialize_bert():
    print("\nInitializing BERT model...")
    try:
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertModel.from_pretrained('bert-base-uncased')
        print("BERT initialized successfully")
        return tokenizer, model
    except Exception as e:
        print(f"Error initializing BERT: {e}")
        return None, None

if __name__ == "__main__":
    FILE_PATH = "data/train.csv/train.csv"

    print("Starting Toxic Comment Classification Pipeline")
    df = load_dataset(FILE_PATH)

    if df is not None:
        df['cleaned_text'] = df['comment_text'].apply(clean_text)
        df['processed_text'] = df['cleaned_text'].apply(remove_stopwords)

        # show some examples of cleaning
        print("\nText processing examples:")
        for i in range(3):
            print(f"\nOriginal {i}:")
            print(df['comment_text'].iloc[i])
            print("\nProcessed:")
            print(df['processed_text'].iloc[i])
            print("-"*50)

        bert_tokenizer, bert_model = initialize_bert()

        if bert_tokenizer:
            print("\nTokenization example:")
            sample_text = df['processed_text'].iloc[0]
            print(f"\nSample text: {sample_text}")

            tokens = bert_tokenizer.tokenize(sample_text)
            print(f"\nTokens: {tokens}")

            token_ids = bert_tokenizer.encode(sample_text, add_special_tokens=True)
            print(f"\nToken IDs: {token_ids}")

    print("\nPipeline execution complete")


C:\Users\aarya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


Starting Toxic Comment Classification Pipeline
Dataset loaded successfully!
Shape: (159571, 8)

Null values per column:
id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

Text processing examples:

Original 0:
Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27

Processed:
explanation edits made username hardcore metallica fan reverted n't vandalisms closure gas voted new york dolls fac please n't remove since retired
--------------------------------------------------

Original 1:
D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)

Processed:
d'aww matches background colour seemingly stuck thanks januar

In [8]:
if df is not None:
    df['cleaned_text'] = df['comment_text'].apply(clean_text)
    df['processed_text'] = df['cleaned_text'].apply(remove_stopwords)


In [9]:
import torch
from tqdm import tqdm
tqdm.pandas()

def get_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
    return cls_embedding.squeeze().numpy()

df['bert_embedding'] = df['processed_text'].progress_apply(
    lambda x: get_bert_embedding(x, bert_tokenizer, bert_model)
)


100%|██████████| 159571/159571 [2:34:44<00:00, 17.19it/s]  


In [10]:
# Show the shape of the embeddings and a preview
print("\nBERT embeddings preview:")
print(df['bert_embedding'].head(3))  # Show first 3



BERT embeddings preview:
0    [-0.31448662, -0.090807594, -0.13584879, -0.26...
1    [-0.64000684, -0.1590128, 0.1361236, -0.327551...
2    [0.06851862, 0.18524176, -0.1254544, -0.105594...
Name: bert_embedding, dtype: object


In [11]:
df[['processed_text', 'bert_embedding']].head(3)


,processed_text,bert_embedding
0,explanation edits made username hardcore metal...,"[-0.31448662, -0.090807594, -0.13584879, -0.26..."
1,d'aww matches background colour seemingly stuc...,"[-0.64000684, -0.1590128, 0.1361236, -0.327551..."
2,hey man really not trying war guy constantly r...,"[0.06851862, 0.18524176, -0.1254544, -0.105594..."


In [12]:
#save the embeddings to csv so we don't have to rerun the code -  don't need to rerun this cell this is for just saving the embeddings
df.to_csv("data/trained_embeddings.csv", index=False)


In [13]:
df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,cleaned_text,processed_text,bert_embedding
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation why the edits made under my userna...,explanation edits made username hardcore metal...,"[-0.31448662, -0.090807594, -0.13584879, -0.26..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,d'aww! he matches this background colour i'm s...,d'aww matches background colour seemingly stuc...,"[-0.64000684, -0.1590128, 0.1361236, -0.327551..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man i'm really not trying to edit war. it'...,hey man really not trying war guy constantly r...,"[0.06851862, 0.18524176, -0.1254544, -0.105594..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,more i can't make any real suggestions on impr...,n't make real suggestions improvement wondered...,"[0.08597994, 0.38976228, 0.22331487, -0.111932..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir are my hero. any chance you remember w...,sir hero chance remember,"[-0.13568623, 0.1466079, -0.08539349, -0.02937..."


In [14]:
df.drop('comment_text', axis=1, inplace=True)  # Drop original text column
df.drop('cleaned_text', axis=1, inplace=True)  # Drop cleaned text column
df.drop('processed_text', axis=1, inplace=True)  # Drop processed text column
df.drop('id', axis=1, inplace=True)  # Drop BERT embedding column
df.head()


,toxic,severe_toxic,obscene,threat,insult,identity_hate,bert_embedding
0,0,0,0,0,0,0,"[-0.31448662, -0.090807594, -0.13584879, -0.26..."
1,0,0,0,0,0,0,"[-0.64000684, -0.1590128, 0.1361236, -0.327551..."
2,0,0,0,0,0,0,"[0.06851862, 0.18524176, -0.1254544, -0.105594..."
3,0,0,0,0,0,0,"[0.08597994, 0.38976228, 0.22331487, -0.111932..."
4,0,0,0,0,0,0,"[-0.13568623, 0.1466079, -0.08539349, -0.02937..."


In [15]:
#train test split

features = ['bert_embedding']
target_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
x= df[features]
y = df[target_columns]

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)   
print(f"Shape of X_train: {x_train.shape}")
print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of X_test: {x_test.shape}")
print(f"Shape of y_test: {y_test.shape}")

Shape of X_train: (127656, 1)
Shape of y_train: (127656, 6)
Shape of X_test: (31915, 1)
Shape of y_test: (31915, 6)


In [ ]:
#use base-uncased model
#take embeddings and make predictions of the target columns

#train a model to predict the target columns using the embeddings

#test the model after training

#linearSVC 
import sklearn
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, PrecisionRecallDisplay
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import StratifiedKFold

from sklearn.multiclass import OneVsRestClassifier

def runMultiLabelSVM(x_train, y_train, x_test, y_test):
    clf = OneVsRestClassifier(svm.SVC())
    clf.fit(x_train, y_train)
    
    train_score = clf.score(x_train, y_train)
    test_score = clf.score(x_test, y_test)


    print("\nGenerating predictions...")
    y_pred_train = clf.predict(x_train)
    y_pred_test = clf.predict(x_test)
    y_score_test = clf.decision_function(x_test)  # Get decision scores for Precision Recall curves (only for test needs to be modified)
    print("Predictions generated.")


    print("OneVsRest SVC accuracy on training data:", train_score)
    print("OneVsRest SVC accuracy on test data:", test_score)
    
    train_accuracy = accuracy_score(y_train, y_pred_train, y_pred_test, average='micro')
    test_accuracy = accuracy_score(y_test, y_pred_test, y_pred_test, average='micro')
    print(f"\n--- Evaluation using accuracy_score ---")
    print(f"Accuracy Score (subset accuracy) on training data: {train_accuracy:.4f}")
    print(f"Accuracy Score (subset accuracy) on test data:    {test_accuracy:.4f}") 

    #precision score
    print(f"\n--- Evaluation using precision_score ---")
    train_prec = precision_score(y_train, y_pred_train, y_pred_test, average='micro')
    test_prec = precision_score(y_test, y_pred_test,y_pred_test, average='micro')
    print(f"Precision Score (subset accuracy) on training data: {train_prec:.4f}")
    print(f"Precision Score (subset accuracy) on test data:    {test_prec:.4f}") 

    #f1 score
    print(f"\n--- Evaluation using f1_score ---")
    train_f1 = f1_score(y_train, y_pred_train, y_pred_test, average='micro')
    test_f1 = f1_score(y_test, y_pred_test, y_pred_test, average='micro')
    print(f"f1 Score (subset accuracy) on training data: {train_f1:.4f}")
    print(f"f1 Score (subset accuracy) on test data:    {test_f1:.4f}")     

    #recall score
    print(f"\n--- Evaluation using recall_score ---")
    train_recall = recall_score(y_train, y_pred_train, average = 'micro')
    test_recall = recall_score(y_test, y_pred_test, average='micro')
    print(f"recall Score (subset accuracy) on training data: {train_recall:.4f}")
    print(f"recall Score (subset accuracy) on test data:    {test_recall:.4f}")     

    
    return train_score, test_score, train_accuracy, test_accuracy, y_test, y_pred_test, y_score_test




#run the model
x_train_stacked = np.vstack(x_train['bert_embedding'])
x_test_stacked = np.vstack(x_test['bert_embedding'])

train_score_m, test_score_m, train_acc, test_acc = runMultiLabelSVM(
    x_train_stacked, y_train, x_test_stacked, y_test
)


print("\n--- Final Summary ---")
train_score = train_acc 
dt_score = test_acc     
print(f"Final Train Accuracy (from accuracy_score): {train_score:.4f}")
print(f"Final Test Accuracy (from accuracy_score):  {dt_score:.4f}")



Generating predictions...
Predictions generated.
OneVsRest SVC accuracy on training data: 0.9149119508679576
OneVsRest SVC accuracy on test data: 0.9116089613034624

--- Evaluation using accuracy_score ---
Accuracy Score (subset accuracy) on training data: 0.9149
Accuracy Score (subset accuracy) on test data:    0.9116

--- Evaluation using precision_score ---


ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted', 'samples'].

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score